In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 
import os

In [2]:
def scheduler(epoch):
    if epoch<= WARMUP_EPOCH:
        lr = INITIAL_LEARNINGRATE *epoch/WARMUP_EPOCH
    
    else:
        lr = INITIAL_LEARNINGRATE * DECAY_RATE**(epoch-WARMUP_EPOCH)
    
    return lr

In [3]:
def BuildModel(ModelName):
    print('--------------Building The Model...--------------')
    if ModelName == 'ResNet152' or ModelName == 'Xception':
        if ModelName == 'ResNet152':
            base_model = tf.keras.applications.ResNet152(include_top=False,
                                                    weights='imagenet',
                                                    input_shape=(HEIGHT,WIDTH,3))
        
        elif ModelName == 'Xception':
            base_model = tf.keras.applications.Xception(include_top=False,
                                                weights='imagenet',
                                                input_shape=(HEIGHT,WIDTH,3))
        
        base_model.trainable = True
        print("\nNumber of layers in the base model: ", len(base_model.layers))
        #print(base_model.layers[0].output_shape)
        x = base_model.output
        #x = tf.keras.layers.GlobalMaxPooling2D()(x)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dense(512,activation='relu')(x)
        x = tf.keras.layers.Dense(64,activation='relu')(x)
        out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model = tf.keras.models.Model(inputs=base_model.input, outputs=out)
    
    elif ModelName == 'EfficientNetB7':
        base_model = tf.keras.applications.EfficientNetB7(include_top=False,
                                                weights='imagenet',
                                                input_shape=(HEIGHT,WIDTH,3))
        base_model.trainable = True
        print("\nNumber of layers in the base model: ", len(base_model.layers))
        #print(base_model.layers[0].output_shape)
        x = base_model.output
        #x = tf.keras.layers.GlobalMaxPooling2D()(x)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(1536,activation='relu')(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dense(512,activation='relu')(x)
        x = tf.keras.layers.Dense(64,activation='relu')(x)
        out = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model = tf.keras.models.Model(inputs=base_model.input, outputs=out)
    
    print('\n--------------Done!--------------')
    return model

In [4]:
def ModelDeployment(model, LearningRate, ModelName,  NbOfEpoch, StepsPerEpoch, ValidationSteps):
    ## Freeze all the layers before the `fine_tune_at` layer
    #for layer in base_model.layers[:fine_tune_at]:
    #    layer.trainable =  False
    print('--------------Deploying the Model...--------------')
    model.compile(loss = 'binary_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=LearningRate),
                  metrics = 'accuracy')
    monitor = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                               min_delta=1e-4, 
                                               patience=10, 
                                               verbose=1, 
                                               mode='min',
                                               restore_best_weights=True)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                        factor=0.5,
                                                        patience=3,
                                                        verbose=1,
                                                        mode='min',
                                                        min_delta=1e-4,
                                                        cooldown=0,
                                                        min_lr=MIN_LR,)

    filepath= ModelName+"-{epoch:02d}-{val_loss:.4f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                    monitor='val_loss', 
                                                    verbose=1, 
                                                    save_best_only=False, 
                                                    save_weights_only=True, 
                                                    mode='min',
                                                    save_freq = 'epoch')
    print('--------------Deployed Successfully--------------')
    print('--------------Training Begins--------------')
    history = model.fit(train_generator, 
                            epochs = NbOfEpoch, 
                            steps_per_epoch = StepsPerEpoch, 
                            validation_data = val_generator, 
                            verbose = 1, 
                            validation_steps = ValidationSteps,
                            callbacks = [monitor,lr_scheduler,checkpoint])
    return history

In [5]:
# Hàm vẽ train & val loss
def HistoryPlotting(history):
    fig,ax = plt.subplots(figsize=(8,5))
    ax.plot(history.history['loss'],label='train',color='blue')
    ax.plot(history.history['val_loss'],label='val',color='green')
    ax.set_title('model loss',fontsize=14)
    ax.set_ylabel('loss')
    ax.set_xlabel('epoch')
    ax.legend(loc='upper right',fontsize=13)

In [ ]:
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 16
train_generator, val_generator = DataGeneration(Height = HEIGHT, 
                                                Width = WIDTH,
                                                BatchSize = BATCH_SIZE)

In [ ]:
tf.keras.backend.clear_session()

# ResNet152

In [7]:
Resnet_based_model = BuildModel(ModelName='ResNet152')
Resnet_based_model.summary()

--------------Building The Model...--------------
234700800/234698864 [==============================] - 2s 0us/step

Number of layers in the base model:  515

--------------Done!--------------
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
#STEPS_PER_EPOCH = 200
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
#VALIDATION_STEPS = 50
LEARNING_RATE = 1e-5
NB_EPOCH = 50
#FINETUNE_LEARNING_RATE = LEARNING_RATE/10
MIN_LR = 1e-8 
#fine_tune_at = 200
Resnet50_history = ModelDeployment(model = Resnet_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'ResNet152',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

# Xception

In [8]:
Xception_based_model = BuildModel(ModelName='Xception')
Xception_based_model.summary()

--------------Building The Model...--------------
83689472/83683744 [==============================] - 1s 0us/step

Number of layers in the base model:  132

--------------Done!--------------
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
#STEPS_PER_EPOCH = 200
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
#VALIDATION_STEPS = 50
LEARNING_RATE = 1e-5
NB_EPOCH = 50
MIN_LR = 1e-8 

Xception_history = ModelDeployment(model = Xception_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'Xception',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

# EfficientNetB7

In [9]:
EfficientNet_based_model = BuildModel(ModelName='EfficientNetB7')
EfficientNet_based_model.summary()

--------------Building The Model...--------------
258080768/258076736 [==============================] - 3s 0us/step

Number of layers in the base model:  813

--------------Done!--------------
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 256, 256, 3)  7           rescaling[0][0]                  
________________________________________________________________________________________________

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
#STEPS_PER_EPOCH = 200
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
#VALIDATION_STEPS = 50
LEARNING_RATE = 1e-5
MIN_LR = 1e-8 
NB_EPOCH = 50

EfficientNet_history = ModelDeployment(model = EfficientNet_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'EfficientNetB7',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)